In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import json
from datetime import datetime

## UPDATE: 22.12.02

- 抓取数据：现在的条件只有开始结束日期的限制了。

------------------------------------------------------

## 22.12.01 记录：

-----------------------------------------------------------

#### 0. Before start，我们需要确定的一些事：

- 选多长时间的数据集？

-------------------------------------------------------

#### 1.重新抓数据：

- 之前的条件：结束时间大于等于9，开始时间小于等于18；

- 现在的条件：结束时间大于9点。

理由：考虑到可能会出现一些前一天18点后开始停车，到第二天早上九点之后走人。e.g.: `21-04-01 21：10：00` 来的，`21-04-02 10：05：00`走的，那么4.2 9：00~10：05这个车位还是被占着的。

#### 2. Overlapping问题：

- 不用loop看情况应该是没问题了，运行速度还可以，心理感觉三分钟左右；

- groupby中的元素会影响结果的数量，详情见***10***.

--------------------------------------------

### 尚未完成的一些：

#### 3. 时间分段问题：

- 时间分段的方法（目前看了徐那一组写在html里的方法，感觉应该是可行的，但要先生成panel）;
- 分几段？（徐是每个小时，alex分了三个时间段）**目前有在考虑 1.数据量大小；2.运行时间等因素。

----------------------------------------------
*包括前3个，数据处理环节

#### 4. 下载整合其他的数据集.

- ACS/OSM/San Francisco Open Data(感觉可以借鉴Alex和徐他们两组的)

#### 5. 生成panel.

- python? R?

----------------------------------------------
*建模·测试环节：

#### 6. Feature enginneering.

- time lag/ amenity/ social economical...

#### 7. Model building.

- OLS/poisson

#### 8. Model 测试、比较（CV）.

- cross validation/ LOOCV (time)

#### 9. Mapping(high quality).

#### 10. RMarkdown 撰写，修改（*12/16中午）.

----------------------------------
*展示环节：

#### 11. APP 界面设计.

#### 12. 定时PPT.(*12/9*)

#### 13. 视频（*12/16中午）

----------------------------------------------

## MENU:

- 1.重新抓了数据，只选择了endtime>=9:

- 2.读取有效post_id,进行valid筛选：

- 3.选出起止的日期

- 4.分别取出起止日期在同一天和不在同一天的表

- 5.提取时间 HH:MM:SS

- 6.hms时间修正

- 7.合并date 和 hms

- 8.去掉修正后开始和结束时间相同的数据

- 9.选取需要的数据，重新整合成一个数据集

- 10.处理overlapping

---------------------------

### 一些表(运行此notebook时可以直接加载的)：

`2021_q2_new.csv`,`2021_q2_new.parquet`: 抓下来的原始数据；

`valid_id.csv`: 有效车位；


### 另一些表(此notebook生成的)：

`park_raw.csv`, `park_raw.parquet`: 处理后的数据集。包含抓下来全部原始信息。

`park1.csv`, `park1.parquet`: 仅包含上一行表的重要信息：post_id, street_block,开始结束时间（此表中的修正时间已改名为：session_start_dt和session_end_dt）。

`1201_result.csv`,`1201_result.parquet`: groupby `post_id`的去overlapping结果。

`1201_result_1.csv`,`1201_result_1.parquet`: groupby `post_id`, `street_block`的去overlapping结果。

---------------------
1.重新抓了数据，只选择了endtime>=9:

In [ ]:
## 抓取数据：
# r = requests.get('https://data.sfgov.org/resource/imvp-dq3v.json?$where=(session_start_dt>="2021-04-01" AND session_start_dt<"2021-07-01" AND date_extract_hh(session_end_dt)>="9")&$limit=7200000')
r = requests.get('https://data.sfgov.org/resource/imvp-dq3v.json?$where=(session_start_dt>="2021-04-01" AND session_start_dt<"2021-07-01")&$limit=7200000')
# content_json = json.loads(r.content)
# df = pd.DataFrame(content_json)

## 保存到本地
# path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\2021_q2_new.csv'
# df.to_csv(path)

In [3]:
date_string_format = '%Y/%m/%d %H:%M:%S %p'
date_parserer = lambda s: pd.to_datetime(s,date_string_format)

path = r'C:/Users/yyp/Desktop/Upenn/MUSA/courses/MUSA5080/Final/data/2021_q2_new.csv'
park = pd.read_csv(
    path, 
    parse_dates=[5,6],
    #date_parser=date_parserer,
    infer_datetime_format=True,
    dayfirst=False,
    cache_dates=True
    )

In [4]:
park

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt
0,0,208673671_4_04012021001513,208-06230,08TH ST 600,CASH,2021-04-01 00:15:13,2021-04-01 00:15:13,NS,0.10
1,1,208673672_4_04012021003636,471-14410,HARRISON ST 1400,CASH,2021-04-01 00:36:36,2021-04-01 00:36:36,NS,0.10
2,2,208673677_4_04012021010437,201-00100,01ST ST 0,CASH,2021-04-01 01:04:37,2021-04-01 01:04:37,NS,1.15
3,3,8186026_11_04012021012801,493-13150,ILLINOIS ST 1300,CREDIT CARD,2021-04-01 01:28:01,2021-04-01 02:12:01,NS,0.40
4,4,8186033_11_04012021020647,700-04560,VALENCIA ST 400,CREDIT CARD,2021-04-01 02:06:47,2021-04-01 02:32:47,NS,2.25
...,...,...,...,...,...,...,...,...,...
5426898,5426898,213193426_4_06302021224018,821-05720,BEALE ST 500,CREDIT CARD,2021-06-30 22:40:18,2021-06-30 23:00:00,NS,0.25
5426899,5426899,213193425_4_06302021224218,830-03120,EMBARCADERO NORTH 300,CREDIT CARD,2021-06-30 22:42:18,2021-06-30 23:00:00,AT,0.29
5426900,5426900,213193433_4_06302021232127,614-07350,POST ST 700,CASH,2021-06-30 23:21:27,2021-06-30 23:21:27,NS,0.25
5426901,5426901,213193435_4_06302021234334,840-22190,GRANT AVE 2200,CASH,2021-06-30 23:43:34,2021-06-30 23:43:34,NS,1.30


-------------------------
2.读取有效post_id,进行valid筛选：

In [5]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\valid_id.csv'

valid_id = pd.read_csv(path)

In [6]:
park['valid'] = park['post_id'].isin(valid_id['post_id'])

park = park.loc[park['valid']==True]

park

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid
0,0,208673671_4_04012021001513,208-06230,08TH ST 600,CASH,2021-04-01 00:15:13,2021-04-01 00:15:13,NS,0.10,True
1,1,208673672_4_04012021003636,471-14410,HARRISON ST 1400,CASH,2021-04-01 00:36:36,2021-04-01 00:36:36,NS,0.10,True
2,2,208673677_4_04012021010437,201-00100,01ST ST 0,CASH,2021-04-01 01:04:37,2021-04-01 01:04:37,NS,1.15,True
20,20,208673692_4_04012021030636,568-31720,MISSION ST 3100,CASH,2021-04-01 03:06:36,2021-04-01 03:06:36,NS,0.05,True
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True
...,...,...,...,...,...,...,...,...,...,...
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True
5426898,5426898,213193426_4_06302021224018,821-05720,BEALE ST 500,CREDIT CARD,2021-06-30 22:40:18,2021-06-30 23:00:00,NS,0.25,True
5426900,5426900,213193433_4_06302021232127,614-07350,POST ST 700,CASH,2021-06-30 23:21:27,2021-06-30 23:21:27,NS,0.25,True
5426901,5426901,213193435_4_06302021234334,840-22190,GRANT AVE 2200,CASH,2021-06-30 23:43:34,2021-06-30 23:43:34,NS,1.30,True


-----------------------
3.选出起止的日期

In [7]:
#开始的日期
park['start_date'] = park['session_start_dt'].dt.date
#结束的日期
park['end_date'] = park['session_end_dt'].dt.date

C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\4286373696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park['start_date'] = park['session_start_dt'].dt.date
C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\4286373696.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park['end_date'] = park['session_end_dt'].dt.date


In [8]:
park

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date
0,0,208673671_4_04012021001513,208-06230,08TH ST 600,CASH,2021-04-01 00:15:13,2021-04-01 00:15:13,NS,0.10,True,2021-04-01,2021-04-01
1,1,208673672_4_04012021003636,471-14410,HARRISON ST 1400,CASH,2021-04-01 00:36:36,2021-04-01 00:36:36,NS,0.10,True,2021-04-01,2021-04-01
2,2,208673677_4_04012021010437,201-00100,01ST ST 0,CASH,2021-04-01 01:04:37,2021-04-01 01:04:37,NS,1.15,True,2021-04-01,2021-04-01
20,20,208673692_4_04012021030636,568-31720,MISSION ST 3100,CASH,2021-04-01 03:06:36,2021-04-01 03:06:36,NS,0.05,True,2021-04-01,2021-04-01
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30
5426898,5426898,213193426_4_06302021224018,821-05720,BEALE ST 500,CREDIT CARD,2021-06-30 22:40:18,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30
5426900,5426900,213193433_4_06302021232127,614-07350,POST ST 700,CASH,2021-06-30 23:21:27,2021-06-30 23:21:27,NS,0.25,True,2021-06-30,2021-06-30
5426901,5426901,213193435_4_06302021234334,840-22190,GRANT AVE 2200,CASH,2021-06-30 23:43:34,2021-06-30 23:43:34,NS,1.30,True,2021-06-30,2021-06-30


-----------------------------
4. 分别取出起止日期在同一天和不在同一天的表

In [9]:
#同一天
park1 = park.loc[park['start_date']==park['end_date']]

In [10]:
park1

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date
0,0,208673671_4_04012021001513,208-06230,08TH ST 600,CASH,2021-04-01 00:15:13,2021-04-01 00:15:13,NS,0.10,True,2021-04-01,2021-04-01
1,1,208673672_4_04012021003636,471-14410,HARRISON ST 1400,CASH,2021-04-01 00:36:36,2021-04-01 00:36:36,NS,0.10,True,2021-04-01,2021-04-01
2,2,208673677_4_04012021010437,201-00100,01ST ST 0,CASH,2021-04-01 01:04:37,2021-04-01 01:04:37,NS,1.15,True,2021-04-01,2021-04-01
20,20,208673692_4_04012021030636,568-31720,MISSION ST 3100,CASH,2021-04-01 03:06:36,2021-04-01 03:06:36,NS,0.05,True,2021-04-01,2021-04-01
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30
5426898,5426898,213193426_4_06302021224018,821-05720,BEALE ST 500,CREDIT CARD,2021-06-30 22:40:18,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30
5426900,5426900,213193433_4_06302021232127,614-07350,POST ST 700,CASH,2021-06-30 23:21:27,2021-06-30 23:21:27,NS,0.25,True,2021-06-30,2021-06-30
5426901,5426901,213193435_4_06302021234334,840-22190,GRANT AVE 2200,CASH,2021-06-30 23:43:34,2021-06-30 23:43:34,NS,1.30,True,2021-06-30,2021-06-30


In [11]:
#不在同一天
park2 = park.loc[park['start_date']!=park['end_date']]

In [12]:
park2

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date
2055914,2055914,8266451_11_05052021145134,700-04320,VALENCIA ST 400,CREDIT CARD,2021-05-05 14:51:34,2021-06-20 01:58:34,NS,2.75,True,2021-05-05,2021-06-20
2056076,2056076,8266455_11_05052021145251,700-04320,VALENCIA ST 400,CREDIT CARD,2021-05-05 14:52:51,2021-06-20 02:00:50,AT,0.25,True,2021-05-05,2021-06-20
5211055,5211055,213017390_4_06262021224506,203-07160,03RD ST 700,CREDIT CARD,2021-06-26 22:45:06,2021-06-27 00:00:00,AT,6.55,True,2021-06-26,2021-06-27


这三条数据都可以删掉。

------------------------
5.提取时间 HH:MM:SS

- update: 只选park1. 

In [14]:
#开始的时间
park1["start_hms"] = park1['session_start_dt'].dt.time

#结束的时间
park1["end_hms"] = park1['session_end_dt'].dt.time

C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\3303382394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park1["start_hms"] = park1['session_start_dt'].dt.time
C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\3303382394.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park1["end_hms"] = park1['session_end_dt'].dt.time


**除去开始时间和结束时间一样的（应在前面完成，此处为补充）

In [23]:
park1 = park1.loc[park1['session_start_dt']<park1['session_end_dt']]

In [24]:
park1

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date,start_hms,end_hms
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01,04:31:04,10:00:00
30,30,208673708_4_04012021044117,201-03130,01ST ST 300,CREDIT CARD,2021-04-01 04:41:17,2021-04-01 13:00:00,NS,9.00,True,2021-04-01,2021-04-01,04:41:17,13:00:00
31,31,208673709_4_04012021044357,831-02180,EMBARCADERO SOUTH 200,CREDIT CARD,2021-04-01 04:43:57,2021-04-01 14:15:40,NS,33.00,True,2021-04-01,2021-04-01,04:43:57,14:15:40
32,32,208673710_4_04012021044837,542-00120,LARKIN ST 0,CREDIT CARD,2021-04-01 04:48:37,2021-04-01 11:00:00,NS,2.00,True,2021-04-01,2021-04-01,04:48:37,11:00:00
35,35,208673712_4_04012021045002,470-05350,HOWARD ST 500,CREDIT CARD,2021-04-01 04:50:02,2021-04-01 11:00:00,NS,13.00,True,2021-04-01,2021-04-01,04:50:02,11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426893,5426893,213193420_4_06302021220131,821-05410,BEALE ST 500,CREDIT CARD,2021-06-30 22:01:31,2021-06-30 23:00:00,NS,0.49,True,2021-06-30,2021-06-30,22:01:31,23:00:00
5426894,5426894,213193421_4_06302021222234,831-02300,EMBARCADERO SOUTH 200,CASH,2021-06-30 22:22:34,2021-06-30 23:00:00,NS,1.05,True,2021-06-30,2021-06-30,22:22:34,23:00:00
5426895,5426895,213193422_4_06302021223105,826-06270,DELANCEY ST 600,CREDIT CARD,2021-06-30 22:31:05,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,22:31:05,23:00:00
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,22:38:10,23:00:00


------------------------------
6.hms时间修正

- 开始时间：小于9的修正为9；大于18的修正为18.
- 结束时间：大于18的修正为18.**小于9的修正为9.

**9点和18点的时间：

In [25]:
nine = dt.time(9,0,0)
six = dt.time(18,0,0)

In [26]:
# 开始时间：小于9的修正为9

park1.loc[park1['start_hms']<nine,'start_hms']=nine

In [27]:
# 开始时间：大于18的修正为18

park1.loc[park1['start_hms']>six,'start_hms']=six

In [28]:
# 结束时间：大于18的修正为18

park1.loc[park1['end_hms']>six,'end_hms']=six

In [29]:
# 结束时间：小于9的修正为9

park1.loc[park1['end_hms']<nine,'end_hms']=nine

In [30]:
park1

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date,start_hms,end_hms
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,10:00:00
30,30,208673708_4_04012021044117,201-03130,01ST ST 300,CREDIT CARD,2021-04-01 04:41:17,2021-04-01 13:00:00,NS,9.00,True,2021-04-01,2021-04-01,09:00:00,13:00:00
31,31,208673709_4_04012021044357,831-02180,EMBARCADERO SOUTH 200,CREDIT CARD,2021-04-01 04:43:57,2021-04-01 14:15:40,NS,33.00,True,2021-04-01,2021-04-01,09:00:00,14:15:40
32,32,208673710_4_04012021044837,542-00120,LARKIN ST 0,CREDIT CARD,2021-04-01 04:48:37,2021-04-01 11:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00
35,35,208673712_4_04012021045002,470-05350,HOWARD ST 500,CREDIT CARD,2021-04-01 04:50:02,2021-04-01 11:00:00,NS,13.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426893,5426893,213193420_4_06302021220131,821-05410,BEALE ST 500,CREDIT CARD,2021-06-30 22:01:31,2021-06-30 23:00:00,NS,0.49,True,2021-06-30,2021-06-30,18:00:00,18:00:00
5426894,5426894,213193421_4_06302021222234,831-02300,EMBARCADERO SOUTH 200,CASH,2021-06-30 22:22:34,2021-06-30 23:00:00,NS,1.05,True,2021-06-30,2021-06-30,18:00:00,18:00:00
5426895,5426895,213193422_4_06302021223105,826-06270,DELANCEY ST 600,CREDIT CARD,2021-06-30 22:31:05,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,18:00:00,18:00:00
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,18:00:00,18:00:00


----------------------------
7.合并date 和 hms
- start: 修正后的开始时间
- end: 修正后的结束时间

In [31]:
# start: 修正后的开始时间

park1['start'] = pd.to_datetime(park1.start_date.astype(str) + ' ' +park1.start_hms.astype(str))

C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\791391419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park1['start'] = pd.to_datetime(park1.start_date.astype(str) + ' ' +park1.start_hms.astype(str))


In [32]:
# end: 修正后的结束时间

park1['end'] = pd.to_datetime(park1.end_date.astype(str) + ' ' +park1.end_hms.astype(str))

C:\Users\yyp\AppData\Local\Temp\ipykernel_19312\2878345148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park1['end'] = pd.to_datetime(park1.end_date.astype(str) + ' ' +park1.end_hms.astype(str))


In [33]:
park1

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date,start_hms,end_hms,start,end
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,10:00:00,2021-04-01 09:00:00,2021-04-01 10:00:00
30,30,208673708_4_04012021044117,201-03130,01ST ST 300,CREDIT CARD,2021-04-01 04:41:17,2021-04-01 13:00:00,NS,9.00,True,2021-04-01,2021-04-01,09:00:00,13:00:00,2021-04-01 09:00:00,2021-04-01 13:00:00
31,31,208673709_4_04012021044357,831-02180,EMBARCADERO SOUTH 200,CREDIT CARD,2021-04-01 04:43:57,2021-04-01 14:15:40,NS,33.00,True,2021-04-01,2021-04-01,09:00:00,14:15:40,2021-04-01 09:00:00,2021-04-01 14:15:40
32,32,208673710_4_04012021044837,542-00120,LARKIN ST 0,CREDIT CARD,2021-04-01 04:48:37,2021-04-01 11:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00,2021-04-01 09:00:00,2021-04-01 11:00:00
35,35,208673712_4_04012021045002,470-05350,HOWARD ST 500,CREDIT CARD,2021-04-01 04:50:02,2021-04-01 11:00:00,NS,13.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00,2021-04-01 09:00:00,2021-04-01 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426893,5426893,213193420_4_06302021220131,821-05410,BEALE ST 500,CREDIT CARD,2021-06-30 22:01:31,2021-06-30 23:00:00,NS,0.49,True,2021-06-30,2021-06-30,18:00:00,18:00:00,2021-06-30 18:00:00,2021-06-30 18:00:00
5426894,5426894,213193421_4_06302021222234,831-02300,EMBARCADERO SOUTH 200,CASH,2021-06-30 22:22:34,2021-06-30 23:00:00,NS,1.05,True,2021-06-30,2021-06-30,18:00:00,18:00:00,2021-06-30 18:00:00,2021-06-30 18:00:00
5426895,5426895,213193422_4_06302021223105,826-06270,DELANCEY ST 600,CREDIT CARD,2021-06-30 22:31:05,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,18:00:00,18:00:00,2021-06-30 18:00:00,2021-06-30 18:00:00
5426897,5426897,213193424_4_06302021223810,818-01880,BRANNAN ST 100,CASH,2021-06-30 22:38:10,2021-06-30 23:00:00,NS,0.25,True,2021-06-30,2021-06-30,18:00:00,18:00:00,2021-06-30 18:00:00,2021-06-30 18:00:00


---------------------
8.去掉修正后开始和结束时间相同的数据

In [34]:
park1 = park1.loc[park1['start']<park1['end']]

In [36]:
park1

,Unnamed: 0,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt,valid,start_date,end_date,start_hms,end_hms,start,end
27,27,208673695_4_04012021043104,440-07110,GEARY ST 700,CASH,2021-04-01 04:31:04,2021-04-01 10:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,10:00:00,2021-04-01 09:00:00,2021-04-01 10:00:00
30,30,208673708_4_04012021044117,201-03130,01ST ST 300,CREDIT CARD,2021-04-01 04:41:17,2021-04-01 13:00:00,NS,9.00,True,2021-04-01,2021-04-01,09:00:00,13:00:00,2021-04-01 09:00:00,2021-04-01 13:00:00
31,31,208673709_4_04012021044357,831-02180,EMBARCADERO SOUTH 200,CREDIT CARD,2021-04-01 04:43:57,2021-04-01 14:15:40,NS,33.00,True,2021-04-01,2021-04-01,09:00:00,14:15:40,2021-04-01 09:00:00,2021-04-01 14:15:40
32,32,208673710_4_04012021044837,542-00120,LARKIN ST 0,CREDIT CARD,2021-04-01 04:48:37,2021-04-01 11:00:00,NS,2.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00,2021-04-01 09:00:00,2021-04-01 11:00:00
35,35,208673712_4_04012021045002,470-05350,HOWARD ST 500,CREDIT CARD,2021-04-01 04:50:02,2021-04-01 11:00:00,NS,13.00,True,2021-04-01,2021-04-01,09:00:00,11:00:00,2021-04-01 09:00:00,2021-04-01 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5425735,5425735,213192561_4_06302021175932,218-34390,18TH ST 3400,CASH,2021-06-30 17:59:32,2021-06-30 18:00:00,NS,0.25,True,2021-06-30,2021-06-30,17:59:32,18:00:00,2021-06-30 17:59:32,2021-06-30 18:00:00
5425737,5425737,213192537_4_06302021175935,684-00860,TOWNSEND ST 0,CREDIT CARD,2021-06-30 17:59:35,2021-06-30 21:59:35,NS,2.00,True,2021-06-30,2021-06-30,17:59:35,18:00:00,2021-06-30 17:59:35,2021-06-30 18:00:00
5425738,5425738,213192549_4_06302021175941,372-06140,CHINA BASIN ST 600,CREDIT CARD,2021-06-30 17:59:41,2021-06-30 20:00:34,NS,1.01,True,2021-06-30,2021-06-30,17:59:41,18:00:00,2021-06-30 17:59:41,2021-06-30 18:00:00
5425739,5425739,213192550_4_06302021175943,681-26130,TAYLOR ST 2600,CREDIT CARD,2021-06-30 17:59:43,2021-06-30 19:00:00,NS,4.53,True,2021-06-30,2021-06-30,17:59:43,18:00:00,2021-06-30 17:59:43,2021-06-30 18:00:00


4110572条数据，和按照开始时间小于18，结束时间大于9筛选出来的数据是一样的。

In [49]:
# 保存：
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\park_raw.csv'
park.to_csv(path1)

path2 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\park_raw.parquet'
park.to_parquet(path2)

## 总结：

在所有valid的数据中，没有出现过夜停车的数据（主要是指，在前一天18点之后开始停车，并且到第二天早上九点之后结束的情况&&前一天18点之前开始停车，并且到第二天早上九点之前结束的情况）。

-------------------------
9. 选取需要的数据，重新整合成一个数据集：

In [38]:
# 需要的列：

cols=['post_id',
     'street_block',
     'start',
     'end']

防止混淆原始数据，新生成的数据表命名为 park1

In [39]:
park1 = park1.loc[:,cols]

In [40]:
park1

,post_id,street_block,start,end
27,440-07110,GEARY ST 700,2021-04-01 09:00:00,2021-04-01 10:00:00
30,201-03130,01ST ST 300,2021-04-01 09:00:00,2021-04-01 13:00:00
31,831-02180,EMBARCADERO SOUTH 200,2021-04-01 09:00:00,2021-04-01 14:15:40
32,542-00120,LARKIN ST 0,2021-04-01 09:00:00,2021-04-01 11:00:00
35,470-05350,HOWARD ST 500,2021-04-01 09:00:00,2021-04-01 11:00:00
...,...,...,...,...
5425735,218-34390,18TH ST 3400,2021-06-30 17:59:32,2021-06-30 18:00:00
5425737,684-00860,TOWNSEND ST 0,2021-06-30 17:59:35,2021-06-30 18:00:00
5425738,372-06140,CHINA BASIN ST 600,2021-06-30 17:59:41,2021-06-30 18:00:00
5425739,681-26130,TAYLOR ST 2600,2021-06-30 17:59:43,2021-06-30 18:00:00


重新命名start,end:

In [41]:
park1.rename({'start': 'session_start_dt', 'end': 'session_end_dt'}, axis=1, inplace=True)

In [42]:
park1

,post_id,street_block,session_start_dt,session_end_dt
27,440-07110,GEARY ST 700,2021-04-01 09:00:00,2021-04-01 10:00:00
30,201-03130,01ST ST 300,2021-04-01 09:00:00,2021-04-01 13:00:00
31,831-02180,EMBARCADERO SOUTH 200,2021-04-01 09:00:00,2021-04-01 14:15:40
32,542-00120,LARKIN ST 0,2021-04-01 09:00:00,2021-04-01 11:00:00
35,470-05350,HOWARD ST 500,2021-04-01 09:00:00,2021-04-01 11:00:00
...,...,...,...,...
5425735,218-34390,18TH ST 3400,2021-06-30 17:59:32,2021-06-30 18:00:00
5425737,684-00860,TOWNSEND ST 0,2021-06-30 17:59:35,2021-06-30 18:00:00
5425738,372-06140,CHINA BASIN ST 600,2021-06-30 17:59:41,2021-06-30 18:00:00
5425739,681-26130,TAYLOR ST 2600,2021-06-30 17:59:43,2021-06-30 18:00:00


------------------------
10. 处理overlapping

- 处理的函数：join_times()

In [38]:
def join_times(x):
    startdf = pd.DataFrame({'time':x["session_start_dt"], 'what':1})
    enddf = pd.DataFrame({'time':x['session_end_dt'], 'what':-1})
    mergdf = pd.concat([startdf, enddf]).sort_values('time')
    mergdf['running'] = mergdf['what'].cumsum()
    mergdf['newwin'] = mergdf['running'].eq(1) & mergdf['what'].eq(1)
    mergdf['group'] = mergdf['newwin'].cumsum()
    x['group'] = mergdf['group'].loc[mergdf['what'].eq(1)]
    res = x.groupby('group').agg({'session_start_dt':'first','session_end_dt':'last' })
    res["interval"] = (res['session_end_dt']-res['session_start_dt']).dt.seconds
    return res

- 检查开始时间是否小于结束时间：

In [43]:
park1 = park1.loc[park1['session_start_dt']<park1['session_end_dt']]

In [44]:
park1

,post_id,street_block,session_start_dt,session_end_dt
27,440-07110,GEARY ST 700,2021-04-01 09:00:00,2021-04-01 10:00:00
30,201-03130,01ST ST 300,2021-04-01 09:00:00,2021-04-01 13:00:00
31,831-02180,EMBARCADERO SOUTH 200,2021-04-01 09:00:00,2021-04-01 14:15:40
32,542-00120,LARKIN ST 0,2021-04-01 09:00:00,2021-04-01 11:00:00
35,470-05350,HOWARD ST 500,2021-04-01 09:00:00,2021-04-01 11:00:00
...,...,...,...,...
5425735,218-34390,18TH ST 3400,2021-06-30 17:59:32,2021-06-30 18:00:00
5425737,684-00860,TOWNSEND ST 0,2021-06-30 17:59:35,2021-06-30 18:00:00
5425738,372-06140,CHINA BASIN ST 600,2021-06-30 17:59:41,2021-06-30 18:00:00
5425739,681-26130,TAYLOR ST 2600,2021-06-30 17:59:43,2021-06-30 18:00:00


- 将session_start_dt,session_end_dt转换成datetime格式：

In [45]:
park1['session_start_dt']=pd.to_datetime(park1['session_start_dt'])

park1['session_end_dt']=pd.to_datetime(park1['session_end_dt'])

- 进行去overlapping:
----------------------------------
此处有一个问题，就是groupby里面放的变量个数会影响最后的结果的数量。

该notebook在之前重新整理数据集的时候，（见9.，生成新表：park1）只选择了post_id和street_block变量，所以问题可能不是很明显。

主要表现在：


1. 仅group by `post_id`，生成 2878234 行结果；

2. 若group by `post_id`和`street_block`，会生成 2878264 行结果。

结果新增了30条。

如果在生成新表的时候多选其他的变量，e.g. 比如增加了 'Month'之类的，会导致结果明显增加（会增加4000+行,来自1201下午的试验）。

---------------------------------------
##### 想法：

考虑是在transaction记录中，有的meter对应了多个street block（可能是记录出错）。

在park1中筛选`post_id`和`street_block`可以发现这个问题：未unique()之前的`post_id`的数量是 15393；unique()之后是 15283. 明显是出现了一对多。

但是包含`Month`的groupby情况不知道怎么解释。

----------------------------------------------

1. 仅group by `post_id`

In [43]:
result = park1.groupby(['post_id']).apply(join_times).reset_index().drop('group', axis=1)

此处的运行时间约为3min.(心理感觉)

In [45]:
result.sort_values('interval')

,post_id,session_start_dt,session_end_dt,interval
2857540,855-00500,2021-06-04 09:00:00,2021-06-04 09:00:01,1
997696,380-03030,2021-05-22 09:00:00,2021-05-22 09:00:01,1
1934084,568-52030,2021-06-01 17:59:59,2021-06-01 18:00:00,1
1744900,546-27210,2021-04-13 17:59:59,2021-04-13 18:00:00,1
976818,372-06300,2021-06-05 17:59:59,2021-06-05 18:00:00,1
...,...,...,...,...
631018,331-04070,2021-05-11 09:00:00,2021-05-11 18:00:00,32400
631025,331-04070,2021-05-15 09:00:00,2021-05-15 18:00:00,32400
2565736,684-00660,2021-06-04 09:00:00,2021-06-04 18:00:00,32400
2143650,593-17210,2021-05-19 09:00:00,2021-05-19 18:00:00,32400


In [50]:
# 保存：
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1201_result.csv'
result.to_csv(path1)

path2 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1201_result.parquet'
result.to_parquet(path2)

2. group by `post_id`和`street_block`

In [47]:
result1 = park1.groupby(['post_id','street_block']).apply(join_times).reset_index().drop('group', axis=1)

In [48]:
result1.sort_values('interval')

,post_id,street_block,session_start_dt,session_end_dt,interval
1934084,568-52030,MISSION ST 5100,2021-06-01 17:59:59,2021-06-01 18:00:00,1
997696,380-03030,DAVIS ST 300,2021-05-22 09:00:00,2021-05-22 09:00:01,1
1898910,568-45330,MISSION ST 4500,2021-04-15 17:59:59,2021-04-15 18:00:00,1
2857570,855-00500,LOMBARD ST 0,2021-06-04 09:00:00,2021-06-04 09:00:01,1
976818,372-06300,CHINA BASIN ST 600,2021-06-05 17:59:59,2021-06-05 18:00:00,1
...,...,...,...,...,...
700970,339-04450,BERRY ST 400,2021-06-11 09:00:00,2021-06-11 18:00:00,32400
1416697,462-17050,HAIGHT ST 1700,2021-06-07 09:00:00,2021-06-07 18:00:00,32400
2581732,686-07300,TURK ST 700,2021-06-30 09:00:00,2021-06-30 18:00:00,32400
2792506,821-05410,BEALE ST 500,2021-04-26 09:00:00,2021-04-26 18:00:00,32400


比`result`多了30行。

In [51]:
# 保存：
path1 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1201_result_1.csv'
result1.to_csv(path1)

path2 = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1201_result_1.parquet'
result1.to_parquet(path2)